## Deployment

Какво е deployment? В изолация, всяко парче код е безполезно, включително и ML/AI. Deployment е процесът по "отваряне" на кода, така че други системи или потребители да могат да достъпват нашия софтуер.

В това демо ние ще:
1. Натренираме модел за оценка на sentiment на филмови коментари на български език
2. Подготвим web service, който да използва натренирания модел да оценява нови ревюта
3. Deploy на дадения web service в облака, за да бъде достъпен

## Трениране на модел
Ще натренираме модел, който използва отворени данни, за да оцени дали дадено ревю за филм е позитивно или негативно (т.нар sentiment). Използваме готов dataset от huggingface - някой се е погрижил да събере ревюта на български и имаме леснодостъпна информация кое е позитивно/негативно.

In [1]:
from datasets import load_dataset
dataset = "sepidmnorozy/Bulgarian_sentiment"
ds = load_dataset(dataset, split="train")
ds['text'][:5]

['Невероятен!!! Ако искате да гледате нещо нестандартно, не се двоумете - това е филмът за вас ;))',
 'Много добре направен :)',
 'Определено най-добрият Хобит!',
 'Муден старт, но нататък е хубав, ненатоварващ и приятен!',
 'Невероятна поредица.']

In [2]:
display(f"Общо имаме {len(ds['label'])} филмови ревюта (samples)")
display(f"Позитивните ревюта са {sum(ds['label'])/len(ds['label'])*100:.2f}% от всички")

'Общо имаме 5412 филмови ревюта (samples)'

'Позитивните ревюта са 84.76% от всички'

Първо, трябва да превърнем текста в ML-readable формат, тоест във вектор от числа. Процесът по преобразуване на текст в числен вектор се нарича tokenization, като всяко число отговаря на token. Примерно, частицата "чно" може да отговаря на числото 346.

In [3]:
from preprocessing import Tokenizer, TokenizingDataset
vocab_size = 350    # number of tokens
tokenizer = Tokenizer(vocab_size)
tokenizer.fit(ds['text'])
train_ds = TokenizingDataset(ds, tokenizer)
train_ds[0]

(tensor([348,  31, 102, 229,  97,  12, 311, 159,  28, 165, 116, 135, 139, 116,
         107, 184, 107,  67, 221, 286,  39,  25,  70,  71,  95, 136,   4, 214,
           2, 164, 116,  18,  32, 115, 142,  61, 216, 114,  83, 178,  23,  36,
          36, 347, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349,
         349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349,
         349, 349, 349, 349, 349, 349, 349, 349, 349, 349, 349],
        dtype=torch.int32),
 tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  

Тренираме модел. Моделът "учи" как изглеждат нашите данни - в случая нашите ревю и какво прави дадено ревю позитивно или негативно. Един научен модел използва "наученото", за да оцени нови ревюта, които не е виждал преди.

In [11]:
from training_procedure import fit_model

model = fit_model(train_ds)

epoch_loss=915.8738434910774
epoch accuracy: 0.7437176644493718
epoch_loss=884.5177562385798
epoch accuracy: 0.844789356984479
epoch_loss=877.668301448226
epoch accuracy: 0.8396156688839616
epoch_loss=860.2318843454123
epoch accuracy: 0.8423872875092387
epoch_loss=898.149313531816
epoch accuracy: 0.8451589061345159
epoch_loss=888.9187539368868
epoch accuracy: 0.8518107908351811
epoch_loss=896.4453482367098
epoch accuracy: 0.8433111603843311
epoch_loss=892.0835991799831
epoch accuracy: 0.8534737620103474
epoch_loss=909.8101763036102
epoch accuracy: 0.8551367331855136
epoch_loss=908.8069994170219
epoch accuracy: 0.8590169992609017
epoch_loss=876.7675055526197
epoch accuracy: 0.8584626755358463
epoch_loss=913.730321733281
epoch accuracy: 0.864560236511456
epoch_loss=892.388779072091
epoch accuracy: 0.8638211382113821
epoch_loss=878.2790558943525
epoch accuracy: 0.8680709534368071
epoch_loss=858.0687375226989
epoch accuracy: 0.8678861788617886
epoch_loss=869.3443309804425
epoch accuracy: 0

За да не се налага да тренираме всеки път, запазваме нашите тренирани компоненти

In [6]:
import torch

tokenizer.save("tokenizer.pkl")
torch.save(model, "model.pt")

За да използваме модела:

In [12]:
model(torch.IntTensor([[10, 15, 20, 25]]))

tensor([[-2.1574,  2.2805]], grad_fn=<AddmmBackward0>)

## Inference
Това е когато използваме модел натрениран върху познати данни, за да даде резултати върху данни, които той вижда за първи път. На практика, това е стъпката, която придава стойност на нашия модел.

In [13]:
import torch
class SentimentInference():
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def __call__(self, inp: str):
        tokens = self.tokenizer.tokenize(inp)[:self.model.context_size+1]
        res = self.model(torch.IntTensor([tokens]))
        return res.argmax().item()  # 0 is negative, 1 is positive
    
    @staticmethod
    def load():
        m = torch.load("model.pt")
        t = Tokenizer.load("tokenizer.pkl")
        return SentimentInference(m, t)
inference = SentimentInference.load()
inference("скучен и глупав филм")

0

In [14]:
inference("филмът беше много готин")

1

## Отваряне на достъпа
Следва да пуснем този модел на прод. Нашето решение ще бъде:
- в облака
- безплатно
- достъпно

In [ ]:
from fastapi import FastAPI
from model import SentimentInference
from pydantic import BaseModel


class Review(BaseModel):
    text: str
    username: str


app = FastAPI()
inf = SentimentInference.load()

@app.post("/sentiment")
async def infer_sentiment(req: Review):
    res = inf(req.text)
    response = "negative" if res == 0 else "positive"
    return {"sentiment": response}

## Отваряне на достъпа
Сега ще отворим безплатен сървър, чиято цел ще бъде да "държи" нашия модел и да може да отговаря на "рекуести". Ще го направим в облака и ще качим на сървъра нашия код, заедно с тренираните ни модели. Ще отворим порт, за да можем да го достъпваме през интернет.

# Какво липсва в тази презентация?
- automated deploy - понякога се налага периодично да тренираме нов модел и той автоматично да се деплойва вместо ръчно да го качваме по всички наши сървъри
- security - всеки може да достъпва нашата виртуална машина. Нямаме никаква защита против атака, примерно DDOS
- scale - моделът е прост и има само една инстанция. Обикновено се налага да имаме възможност да осигурим повече ресурси
- docker и kubernetes - един сървиз би влязъл в контейнер под kubernetes. На практика, няма да се налага да правим ръчните команди по сетване на нова виртуалка, а нашия сървиз би бил по-secure и по-manageable откъм monitoring, logging, automated scaling, etc
- stress test - на какъв брой рекуести може да издържи нашия сървиз
- monitoring - как да разберем възможно най-бързо когато има проблем със сървиза или AWS
- networking - важна тема, но реших да не разводнявам дискусията. Добра идея е поне да фиксираме адреса на нашия сървиз, така че да не го викаме по IP